In [218]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/train.csv', index_col=None)

In [219]:
test = pd.read_csv('data/test.csv', index_col=None)

In [220]:
data1 = df[(df.Sex=='female') & (df.Pclass == 1)]
data1['Age'] = data1['Age'].fillna(35.0)
data2 = df[(df.Sex=='female') & (df.Pclass == 2)]
data2['Age'] = data2['Age'].fillna(28.0)
data3 = df[(df.Sex=='female') & (df.Pclass == 3)]
data3['Age'] = data3['Age'].fillna(21.5)
data4 = df[(df.Sex=='male') & (df.Pclass == 1)]
data4['Age'] = data4['Age'].fillna(40.0)
data5 = df[(df.Sex=='male') & (df.Pclass == 2)]
data5['Age'] = data5['Age'].fillna(30.0)
data6 = df[(df.Sex=='male') & (df.Pclass == 3)]
data6['Age'] = data6['Age'].fillna(25.0)
final_data = pd.concat([data1,data2,data3,data4,data5,data6])
final_data.describe()

C:\Users\Matt\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Matt\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Matt\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.112424,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.304424,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,21.500000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,26.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,36.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [221]:
def names(train, test):
    for i in [train, test]:
        i['Name_Len'] = i['Name'].apply(lambda x: len(x))
        i['Name_Title'] = i['Name'].apply(lambda x: x.split(',')[1]).apply(lambda x: x.split()[0])
        del i['Name']
    return train, test

In [222]:
def age_impute(train, test):
    for i in [train, test]:
        i['Age_Null_Flag'] = i['Age'].apply(lambda x: 1 if pd.isnull(x) else 0)
        data = train.groupby(['Name_Title', 'Pclass'])['Age']
        i['Age'] = data.transform(lambda x: x.fillna(x.mean()))
    return train, test

In [223]:
def fam_size(train, test):
    for i in [train, test]:
        i['Fam_Size'] = np.where((i['SibSp']+i['Parch']) == 0 , 'Solo',
                           np.where((i['SibSp']+i['Parch']) <= 3,'Nuclear', 'Big'))
        del i['SibSp']
        del i['Parch']
    return train, test

In [224]:
def ticket_grouped(train, test):
    for i in [train, test]:
        i['Ticket_Lett'] = i['Ticket'].apply(lambda x: str(x)[0])
        i['Ticket_Lett'] = i['Ticket_Lett'].apply(lambda x: str(x))
        i['Ticket_Lett'] = np.where((i['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), i['Ticket_Lett'],
                                   np.where((i['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            'Low_ticket', 'Other_ticket'))
        i['Ticket_Len'] = i['Ticket'].apply(lambda x: len(x))
        del i['Ticket']
    return train, test

In [225]:
def cabin(train, test):
    for i in [train, test]:
        i['Cabin_Letter'] = i['Cabin'].apply(lambda x: str(x)[0])
        del i['Cabin']
    return train, test

In [226]:
def cabin_num(train, test):
    for i in [train, test]:
        i['Cabin_num1'] = i['Cabin'].apply(lambda x: str(x).split(' ')[-1][1:])
        i['Cabin_num1'].replace('an', np.NaN, inplace = True)
        i['Cabin_num1'] = i['Cabin_num1'].apply(lambda x: int(x) if not pd.isnull(x) and x != '' else np.NaN)
        i['Cabin_num'] = pd.qcut(train['Cabin_num1'],3)
    train = pd.concat((train, pd.get_dummies(train['Cabin_num'], prefix = 'Cabin_num')), axis = 1)
    test = pd.concat((test, pd.get_dummies(test['Cabin_num'], prefix = 'Cabin_num')), axis = 1)
    del train['Cabin_num']
    del test['Cabin_num']
    del train['Cabin_num1']
    del test['Cabin_num1']
    return train, test

In [227]:
def embarked_impute(train, test):
    for i in [train, test]:
        i['Embarked'] = i['Embarked'].fillna('S')
    return train, test

In [228]:
test['Fare'].fillna(final_data['Fare'].mean(), inplace = True)

In [229]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [230]:
def dummies(train,test,columns=['Pclass', 'Sex', 'Embarked', 'Ticket_Lett', 'Cabin_Letter', 'Name_Title', 'Fam_Size']):
    for column in columns:
        train[column] = train[column].apply(lambda x:str(x))
        test[column] = test[column].apply(lambda x:str(x))
        good_cols = [column + "_" + i for i in train[column].unique() if i in test[column].unique()]
        train = pd.concat((train,pd.get_dummies(train[column],prefix=column)[good_cols]),axis=1)
        train = pd.concat((test,pd.get_dummies(test[column],prefix=column)[good_cols]),axis=1)
        del train[column]
        del test[column]
    return train,test

In [231]:
def drop(train, test, bye = ['PassengerId']):
    for i in [train, test]:
        for z in bye:
            del i[z]
    return train, test

In [232]:
final_data, test = names(final_data, test)
final_data, test = age_impute(final_data, test)
final_data, test = cabin_num(final_data, test)
final_data, test = cabin(final_data, test)
final_data, test = embarked_impute(final_data, test)
final_data, test = fam_size(final_data, test)
final_data['Fare'].fillna(final_data['Fare'].mean(), inplace = True)
final_data, test = ticket_grouped(final_data, test)


In [233]:
final_data.head()
test.head()
#final_data.to_csv('data/final_data.csv', index_col=None)
#test.to_csv('data/final_test_data.csv', index_col=None)

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Name_Len,Name_Title,Age_Null_Flag,"Cabin_num_[2, 28.667]","Cabin_num_(28.667, 65.667]","Cabin_num_(65.667, 148]",Cabin_Letter,Fam_Size,Ticket_Lett,Ticket_Len
0,892,3,male,22.0,7.8292,Q,16,Mr.,0,0.0,0.0,0.0,n,Solo,3,6
1,893,3,female,38.0,7.0000,S,32,Mrs.,0,0.0,0.0,1.0,n,Nuclear,3,6
2,894,2,male,26.0,9.6875,Q,25,Mr.,0,0.0,0.0,0.0,n,Solo,2,6
3,895,3,male,35.0,8.6625,S,16,Mr.,0,0.0,0.0,1.0,n,Solo,3,6
4,896,3,female,35.0,12.2875,S,44,Mrs.,0,0.0,0.0,0.0,n,Nuclear,3,7


In [234]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Name_Len,Name_Title,Age_Null_Flag,"Cabin_num_[2, 28.667]","Cabin_num_(28.667, 65.667]","Cabin_num_(65.667, 148]",Cabin_Letter,Fam_Size,Ticket_Lett,Ticket_Len
0,892,3,male,22.0,7.8292,Q,16,Mr.,0,0.0,0.0,0.0,n,Solo,3,6
1,893,3,female,38.0,7.0000,S,32,Mrs.,0,0.0,0.0,1.0,n,Nuclear,3,6
2,894,2,male,26.0,9.6875,Q,25,Mr.,0,0.0,0.0,0.0,n,Solo,2,6
3,895,3,male,35.0,8.6625,S,16,Mr.,0,0.0,0.0,1.0,n,Solo,3,6
4,896,3,female,35.0,12.2875,S,44,Mrs.,0,0.0,0.0,0.0,n,Nuclear,3,7


In [235]:
#final_data, test = dummies(final_data, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett','Cabin_Letter', 'Name_Title', 'Fam_Size'])
#final_data, test = drop(final_data, test)

In [237]:
final_data, test = dummies(final_data, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett','Cabin_Letter', 'Name_Title', 'Fam_Size'])
#final_data, test = drop(final_data, test)
final_data.head()

,PassengerId,Age,Fare,Name_Len,Age_Null_Flag,"Cabin_num_[2, 28.667]","Cabin_num_(28.667, 65.667]","Cabin_num_(65.667, 148]",Ticket_Len,Fam_Size_Solo,Fam_Size_Nuclear,Fam_Size_Big
0,892,22.0,7.8292,16,0,0.0,0.0,0.0,6,1.0,0.0,0.0
1,893,38.0,7.0000,32,0,0.0,0.0,1.0,6,0.0,1.0,0.0
2,894,26.0,9.6875,25,0,0.0,0.0,0.0,6,1.0,0.0,0.0
3,895,35.0,8.6625,16,0,0.0,0.0,1.0,6,1.0,0.0,0.0
4,896,35.0,12.2875,44,0,0.0,0.0,0.0,7,0.0,1.0,0.0
